# DALLE2

[李沐讲论文](https://www.youtube.com/watch?v=hO57mntSMl0&list=RDCMUC8WCW6C3BWLKSZ5cMzD8Gyw&index=16)

![](01.png)

1. CLIP 模型可以获取的图片和文字的特征表示，之前的模型都是用这些特征来做分类任务。而这篇文章讲这个特征用在了生成任务

![](02.png)

1. 上半部分就是CLIP模型，用文本编码器和图像编码器分别得到文本特征和图像特征，将匹配的文字和图片作为positive对，不匹配的作为negative对，计算文字特征和图像特征的内积作为匹配度，最大化交叉熵使得生成的特征向量，匹配对的内积数值高，不匹配的低。这样就建立了图像和文本的关系

2. 下半部分是先用训练好的CLIP模型的text encoder 生成文本特征，img encoder生成图像特征，用这样的pair去训练中间的prior模型，从而使prior模型可以将文本特征转换为图像特征

3. 在应用过程中，输入文字，通过text encoder生成文本特征，用prior模型转换为图像特征，再用decoder将图像特征生成图片

## 图像生成模型串讲


### 1. GAN

![](03.png)

1. 一个生成网络 G 和一个判别网络 D。生成网络输入随机噪声Z，输出一个图片。判别网络输入一个真实的图片和G生成的图片，输出那一个是真实的图片。两个网络对抗学习，最终使得 G 生成的图片非常逼真

2. GAN 的问题是其同时训练两个网络，容易训练不稳定。同时生成的图片单一

### 2. AE（auto encoder），DAE（denoise audo encoder）

![](04.png)

1. 输入一个图片X，通过编码器E将其转换为维度较小的特征Z，再通过解码器D，将其还原为图片X1,目标函数就是X1和X尽量接近

2. DAE 就是将输入认为加入了一些噪音生成Xc，然后将Xc传给Encoder。这样使得模型更稳健，不容易过拟合

### VAE

![](05.png)


1. VAE 和 AE 的区别在于其中间生成的不是一个特征向量，而是一个正态分布$(\mu,\sigma)$，然后通过对这个正态分布采样生成一个特征向量Z，对Z用解码器变成X‘，目标也是让X’与X一样

2. 因为VAE生成的是个概率分布，从其中抽样的Z就有多样性，从而使得结果也具有多样性，可以用在生成模型中

### VQVAE （vector quantised VAE)

![](06.png)

1. 根VAE类似，只不过中间不是预测一个分布，而是用一个固定的codebook代替。codebook是一个KxD的矩阵，每一列是一个特征向量，其实我理解就相当于一些基向量。输入的图像X经过编码器输出一个特征图f，将f中的向量与这些特征图中的向量对比，找出最相似的一些组成中间矩阵Z（Z是选中的基向量的权重矩阵），然后根据Z从codebook向量中构造新的特征图fa，将其输入到解码器D，输出X‘

2. 因为codebook是固定的，因此训练就相对稳定容易

### DALL-E

![](07.png)

1. 将文本用BPE编码成文字特征ft
2. 将图片用训练好的VQVAE编码成图像特征fq
3. 训练的时候将两个特征concat到一起，输入到GPT中预测下一个token，训练好GPT
4. 生成的时候，只提供文本特征ft，用GPT去预测图像特征，然后再用VQVAE的decoder把图像特征fq还原成图片